## Overview for 4/12/22 Meeting
The below code offers an overview of the GGIR R package, and, more importantly, runs a sample processing pipeline on a single participant from AHAB. The general structue of the below GGIR code is stable but small details/specifications are likely to update as we get closer to being ready to screen. 



## Basics of GGIR

[GGIR](https://CRAN.R-project.org/package=GGIR) is an R-package to process multi-day raw accelerometer data for physical activity and sleep research. The signal processing includes automatic calibration, detection of sustained abnormally high values, detection of non-wear and calculation of average magnitude of dynamic acceleration based on a variety of metrics. Next, GGIR uses this information to describe the data per recording, per day of measurement, and (optionally) per segment of a day of measurement, including estimates of physical activity, inactivity and sleep.
See overview paper of GGIR from 2019: [link](https://doi.org/10.1123/jmpb.2018-0063).


GGIR works with with many accelerometer brands and formats, including ActiGraph. [ActiGraph](https://actigraphcorp.com/) .csv and .gt3x files are both compatable (the below code uses the .gt3x file)

All accelerometer data that needs to be analysed should be stored in one folder, or subfolders of that folder.


## GGIR shell function

There are 5 key parts relevant for processing and scoring the actigraphy data in GGIR:
- Part 1: Loads the data and stores derived features (aggregations) needed for the other parts. Delects nonware times. This is the time-consuming part (80% of processing time taken up in this step) 
- Part 2: Data quality analyses and low-level description of signal features per day and per file. At this point a day is defined from midnight to midnight
- Part 3: Estimation of sustained inactivity and sleep periods, needed for input to Part 4 for sleep detection
- Part 4: Labels the sustained inactive periods detected in Part 3 as sleep, or daytime sustained inactivity, per night and per file
- Part 5: Derives sleep and physical activity characteristics by re-using information derived in part 2, 3 and 4. Total time in intensity categories, the number of bouts, time spent in bouts and average acceleration (overall activity) is calculated.

The reason why it split up in parts is that it avoids having the re-do all analysis if you only want to make a small change in the more downstream parts. The specific order and content of the parts has grown for historical and computational reasons.

To ease interacting with GGIR there is one central function, named `g.shell.GGIR`, that incorporates all other functions.

## Example Actigraphy Processing with one AHAB Participant

In [1]:
library(GGIR)

# Running the GGIR shell
mode= c(1,2,3,4,5)
data = "/Users/david/Documents/GitHub_Repos/GGIR_Actigraphy_Scoring/Actigraphy_Processing/Sample Data"
outputdir = "/Users/david/Documents/GitHub_Repos/GGIR_Actigraphy_Scoring/Actigraphy_Processing/Sample Results"             

g.shell.GGIR(
             mode=mode,
             datadir=data,
             outputdir=outputdir,
             do.report=c(2,4,5),
             overwrite = TRUE,        #Overwrites milestone files if they already exist
             do.enmo= TRUE,
             #=====================
             # Part 2
             #=====================
             strategy = 1,            #How to deal with study protocol knowledge, 1 = data based on hrs.del.start and hrs.del.end.
             hrs.del.start = 0,       #Numeric, how many HOURS after start of experiment did wearing of monitor start?       
             hrs.del.end = 0,
             maxdur = 0,              #Numeric, How many DAYS after start of experiment did experiment stop? (set to zero if unknown = default)             
             includedaycrit = 16,     #Numeric, minimum required number of valid hours in day specific analysis 
             qwindow=c(0,24),         #Numeric or character, window over which variables are calculated. c(0,24) = full 24 hours/day
             mvpathreshold =c(100.6), #Numeric, Acceleration threshold for MVPA estimation 
             bout.metric = 6,         #Numeric, Specify a metric for bout detection.
             excludefirstlast = FALSE,
             includenightcrit = 16,   #Numeric. Minimum number of valid hours per night (24 hour window between noon and noon),
             #=====================
             # Part 3 + 4
             #=====================
             anglethreshold = 5,      #Numeric, Angle threshold (degrees) for sustained inactivity periods detection
             timethreshold = 5,       #Numeric, time threshold (minutes) for sustained inactivity periods detection
             def.noc.sleep = 1,       #Numeric. The time window during which sustained inactivity will be assumed to represent sleep
             outliers.only = TRUE,    #Boolean.  Outliers.only = TRUE will visualise only for nights with a difference in onset or waking time larger than the variable of argument criterror
             criterror = 4,           #Numeric. For outliets.only, specifies minimum number of hours difference between sleep log and accelerometer estimate for the night to be included in the visualisation
             do.visual = TRUE,        #do.visual == TRUE generates pdf with a visual representation of the overlap between the sleeplog entries and the accelerometer detections.
             #=====================
             # Part 5
             #=====================
             threshold.lig = c(48.8), #Numeric. Thresholds for light, medium, and vigerous physical activity
             threshold.mod = c(100.6),  
             threshold.vig = c(428.8),
             boutcriter = 0.8,        #Numeric. A number between 0 and 1 and defines what fraction of a bout needs to be:
             boutcriter.in = 0.9,              # below the light threshold
             boutcriter.lig = 0.8,             # between light and medium
             boutcriter.mvpa = 0.8,            # above mvpathreshold
             boutdur.in = c(1,10,30), #Numeric, see Durations of inactivty bouts in minutes to be extracted.
             boutdur.lig = c(1,10),   #Numeric, see Durations of light activty bouts in minutes to be extracted
             boutdur.mvpa = c(1),     #Numeric, see Durations of mvpa bouts in minutes to be extracted.
             includedaycrit.part5 = 2/3,
             #=====================
             # Visual report
             #=====================
             timewindow = c("WW"),
             visualreport=TRUE)


Warning message:
“package ‘GGIR’ was built under R version 4.1.2”



   GGIR version:  2.6-0

   Do not forget to cite GGIR in your publications via a version number and
   Migueles et al. 2019 JMPB. doi: 10.1123/jmpb.2018-0063. 
   See also: https://cran.r-project.org/package=GGIR/vignettes/GGIR.html#citing-ggir

   To make your research reproducible and interpretable always report:
     (1) Accelerometer brand and product name
     (2) How you configured the accelerometer
     (3) Study protocol and wear instructions given to the participants
     (4) GGIR version
     (5) How GGIR was used: Share the config.csv file or your R script.
     (6) How you post-processed / cleaned GGIR output
     (7) How reported outcomes relate to the specific variable names in GGIR
________________________________________________________________________________
 Part 1 

Checking that user has read access permission for all files in data directory: Yes
Checking that user has write access permission for directory specified by argument outputdir: Yes

 Busy processing ..